In [1]:
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt 
from multiprocessing import Pool
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
### read census tract geometry
census_tracts_gdf = gpd.read_file('traffic_inputs/census_tract.geojson')
census_tracts_gdf = census_tracts_gdf.to_crs(4326)
ba_census_tracts_gdf = census_tracts_gdf.loc[census_tracts_gdf['COUNTY'].isin(['001', '013', '041', '055', '075', '081', '085', '095', '097'])].copy().reset_index(drop=True)
# ba_census_tracts_gdf['FIPS'] = ba_census_tracts_gdf['GEO_ID'].apply(lambda x: int(x.split('US')[1]))
ba_census_tracts_gdf['FIPS'] = ba_census_tracts_gdf['GEO_ID'].apply(lambda x: x.split('US')[1])
print(ba_census_tracts_gdf.shape)
ba_census_tracts_gdf.head(2)

(1585, 9)


,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,geometry,FIPS
0,1400000US06001425103,06,001,425103,4251.03,Tract,0.341,"MULTIPOLYGON (((-122.29236 37.84936, -122.2895...",06001425103
1,1400000US06001425104,06,001,425104,4251.04,Tract,0.415,"MULTIPOLYGON (((-122.27860 37.82855, -122.2780...",06001425104


In [3]:
### building
building_df = pd.read_csv('BayArea-Buildings-Damage-3.csv')
building_gdf = gpd.GeoDataFrame(building_df, crs=4326, geometry=[Point(xy) for xy in zip(building_df.longitude, building_df.latitude)])
building_gdf.head(2)

,0,1,2,3,4,5,6,7,8,9,...,latitude,stories,yearbuilt,occupancy,structure,areafootprint,replacementCost,population,MSID,geometry
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,37.716307,1,1948,RES1,W1,90.423164,1.0,1,9794592,POINT (-122.49505 37.71631)
1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,37.716702,1,1947,RES1,W1,187.357970,1.0,1,9794593,POINT (-122.47831 37.71670)


In [22]:
building_threshold = 5
building_gdf.loc[building_gdf['0']>=building_threshold].shape

(0, 112)

In [23]:
def building_damage_ratio(tract_info):
    [tract_id, tract_geom] = tract_info
    coarse_match_ids = list(building_sindex.intersection(tract_geom.bounds))
    coarse_matchs = building_gdf.iloc[coarse_match_ids]
    precise_matches = coarse_matchs[coarse_matchs.intersects(tract_geom)]
    damage_cnt = [tract_id, precise_matches.shape[0]]
    for i in range(100):
        damage_cnt.append(precise_matches.loc[precise_matches['{}'.format(i)]>=building_threshold].shape[0])
    return damage_cnt

building_sindex = building_gdf.sindex
print('finish spatial indexing')
pool = Pool(processes=5)
tract_info_list = ba_census_tracts_gdf[['GEO_ID', 'geometry']].values.tolist()
res = pool.imap_unordered(building_damage_ratio, tract_info_list)
pool.close()
pool.join()

tract_building_damage_list = [tract_res for tract_res in res]
tract_building_damage_df = pd.DataFrame(tract_building_damage_list, columns=['GEO_ID', 'total_buildings'] + ['damaged_bld_cnts_rep{}'.format(i) for i in range(100)])
tract_building_damage_df.head(2)

finish spatial indexing


,GEO_ID,total_buildings,damaged_bld_cnts_rep0,damaged_bld_cnts_rep1,damaged_bld_cnts_rep2,damaged_bld_cnts_rep3,damaged_bld_cnts_rep4,damaged_bld_cnts_rep5,damaged_bld_cnts_rep6,damaged_bld_cnts_rep7,...,damaged_bld_cnts_rep90,damaged_bld_cnts_rep91,damaged_bld_cnts_rep92,damaged_bld_cnts_rep93,damaged_bld_cnts_rep94,damaged_bld_cnts_rep95,damaged_bld_cnts_rep96,damaged_bld_cnts_rep97,damaged_bld_cnts_rep98,damaged_bld_cnts_rep99
0,1400000US06001425103,317,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1400000US06001427800,1207,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
### pipe_nodes
### PEER presentation
# pipe_gdf = gpd.read_file('lack_demand_nodes.geojson')
# display(pipe_gdf.head(2))
### PEER report
pipe_gdf = gpd.read_file('supply_shrt_ratio.geojson')
display(pipe_gdf.head(2))

,node_id,longitude,latitude,elevation,demand,MM_2018_gp,im_rep31_shrt_ratio,im_rep71_shrt_ratio,im_rep13_shrt_ratio,im_rep89_shrt_ratio,...,im_rep99_shrt_ratio,im_rep65_shrt_ratio,im_rep78_shrt_ratio,im_rep45_shrt_ratio,im_rep55_shrt_ratio,im_rep29_shrt_ratio,im_rep54_shrt_ratio,im_rep83_shrt_ratio,im_rep58_shrt_ratio,geometry
0,28989,-122.231164,37.772601,17.95,0.0,0.126254,7.838119e-09,1.362566e-08,1.423329e-08,1.206641e-08,...,6.052965e-02,6.819959e-09,1.244053e-08,7.015809e-09,9.153809e-09,1.199988e-08,1.049210e-08,1.218443e-08,1.012301e-08,POINT (-122.23116 37.77260)
1,28991,-122.230007,37.784708,53.43,0.0,0.580767,9.474165e-09,1.216789e-08,1.260729e-08,1.139006e-08,...,7.719288e-09,9.520562e-09,1.163023e-08,9.233996e-09,1.037207e-08,1.142281e-08,1.075689e-08,1.137915e-08,1.065591e-08,POINT (-122.23001 37.78471)


In [24]:
pipe_threshold = 100
pipe_gdf.loc[pipe_gdf['im_rep0_shrt_ratio']>=pipe_threshold].shape

(0, 107)

In [25]:
def pipe_damage_cnt(tract_info):
    [tract_id, tract_geom] = tract_info
    coarse_match_ids = list(pipe_sindex.intersection(tract_geom.bounds))
    coarse_matchs = pipe_gdf.iloc[coarse_match_ids]
    precise_matches = coarse_matchs[coarse_matchs.intersects(tract_geom)]
    damage_cnt = [tract_id]
    for i in range(100):
        damage_cnt.append(precise_matches.loc[precise_matches['im_rep{}_shrt_ratio'.format(i)]>=pipe_threshold].shape[0])
    return damage_cnt

pipe_sindex = pipe_gdf.sindex
print('finish spatial indexing')
pool = Pool(processes=4)
tract_info_list = ba_census_tracts_gdf[['GEO_ID', 'geometry']].values.tolist()
res = pool.imap_unordered(pipe_damage_cnt, tract_info_list)
pool.close()
pool.join()

# tract_pipe_damage_list = [(tract_id, damaged) for (tract_id, damaged) in res]
# tract_pipe_damage_df = pd.DataFrame(tract_pipe_damage_list, columns=['GEO_ID', 'damaged_pipe_cnts'])
tract_pipe_damage_list = [tract_res for tract_res in res]
tract_pipe_damage_df = pd.DataFrame(tract_pipe_damage_list, columns=['GEO_ID'] + ['damaged_pipe_cnts_rep{}'.format(i) for i in range(100)])
tract_pipe_damage_df.head(2)

finish spatial indexing


,GEO_ID,damaged_pipe_cnts_rep0,damaged_pipe_cnts_rep1,damaged_pipe_cnts_rep2,damaged_pipe_cnts_rep3,damaged_pipe_cnts_rep4,damaged_pipe_cnts_rep5,damaged_pipe_cnts_rep6,damaged_pipe_cnts_rep7,damaged_pipe_cnts_rep8,...,damaged_pipe_cnts_rep90,damaged_pipe_cnts_rep91,damaged_pipe_cnts_rep92,damaged_pipe_cnts_rep93,damaged_pipe_cnts_rep94,damaged_pipe_cnts_rep95,damaged_pipe_cnts_rep96,damaged_pipe_cnts_rep97,damaged_pipe_cnts_rep98,damaged_pipe_cnts_rep99
0,1400000US06001426100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1400000US06001425104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
### Find nodes inside each tract
def nodes_in_tract(tract_info):
    [tract_id, tract_geom] = tract_info
    coarse_match_ids = list(nodes_sindex.intersection(tract_geom.bounds))
    coarse_matchs = nodes_gdf.iloc[coarse_match_ids]
    precise_matches = coarse_matchs[coarse_matchs.intersects(tract_geom)]
    return (tract_id, precise_matches['osmid'].values.tolist())

nodes_df = pd.read_csv('../network_inputs/unique_id_nodes.csv')
display(nodes_df.head(2))
nodes_gdf = gpd.GeoDataFrame(nodes_df, crs=4326, geometry=[Point(xy) for xy in zip(nodes_df.lon, nodes_df.lat)])
nodes_sindex = nodes_gdf.sindex
print('finish spatial indexing')
pool = Pool(processes=5)
tract_info_list = ba_census_tracts_gdf[['FIPS', 'geometry']].values.tolist()
res = pool.imap_unordered(nodes_in_tract, tract_info_list)
pool.close()
pool.join()

tract_nodes_dict = {tract_id: nodes_list for (tract_id, nodes_list) in res}
print(len([i for v in tract_nodes_dict.values() for i in v]), nodes_gdf.shape)

,node_id_igraph,osmid,lon,lat
0,0,56098817,-122.769449,38.435336
1,1,65536002,-122.136780,37.407134


finish spatial indexing
209417 (224223, 5)


In [13]:
### read commute data
# commute_data = pd.read_csv('traffic_inputs/commute_data.csv')
commute_data = pd.read_csv('ctpp/ctpp_B302201_bay_area_mode_time.csv', dtype={'origin_state': str, 'origin_county': str, 'origin_tract': str, 'destin_state': str, 'destin_county': str, 'destin_tract': str})
commute_data['OFIPS'] = commute_data.apply(lambda x: x['origin_state']+x['origin_county']+x['origin_tract'], axis=1)
commute_data['DFIPS'] = commute_data.apply(lambda x: x['destin_state']+x['destin_county']+x['destin_tract'], axis=1)
commute_data['FLOW'] = commute_data['vehicles']
commute_data = commute_data[['OFIPS', 'DFIPS', 'FLOW']]
commute_data.head()

## filter to those within Bay Area
ba_commute_data = commute_data.loc[(
    commute_data['OFIPS'].isin(ba_census_tracts_gdf['FIPS'])) & (
    commute_data['DFIPS'].isin(ba_census_tracts_gdf['FIPS']))].reset_index()
print(commute_data.shape, ba_commute_data.shape)
print('bay area has {} census tracts, but only {}/{} census tract level OD pairs, {} trips'.format(
    ba_census_tracts_gdf.shape, len(np.unique(ba_commute_data['OFIPS'])), len(np.unique(ba_commute_data['DFIPS'])), np.sum(ba_commute_data['FLOW'])))
ba_commute_data.head(2)

(100158, 3) (100158, 4)
bay area has (1585, 9) census tracts, but only 1577/1577 census tract level OD pairs, 2049284.5 trips


,index,OFIPS,DFIPS,FLOW
0,0,06001400100,06001400500,20.0
1,1,06001400100,06001401000,10.0


In [26]:
def calculate_flow_after_damage(eq_i, building_threshold, pipe_threshold, damaged_tract_data=None):
    ### After damage: two buildings per water node
    damaged_tract_data['damage_ratio'] = (damaged_tract_data['damaged_bld_cnts_rep{}'.format(eq_i)] + damaged_tract_data['damaged_pipe_cnts_rep{}'.format(eq_i)]*2
    )/damaged_tract_data['total_buildings']
    damaged_tract_data['damage_ratio'] = np.where(damaged_tract_data['damage_ratio']>1, 1, damaged_tract_data['damage_ratio'])
    # damaged_tract_data.to_csv('tract_damage_ratio_eq{}_bld{}_pip{}.csv'.format(eq_i, building_threshold, pipe_threshold), index=False)
    # return

    damaged_commute_data = ba_commute_data[['OFIPS', 'DFIPS', 'FLOW']].copy()
    damaged_commute_data = damaged_commute_data.merge(
        damaged_tract_data[['FIPS', 'damage_ratio']], how='left', left_on='OFIPS', right_on='FIPS').merge(
        damaged_tract_data[['FIPS', 'damage_ratio']], how='left', left_on='DFIPS', right_on='FIPS', suffixes=['_O', '_D'])

    # damaged_commute_data['flow_n'] = damaged_commute_data['FLOW']
    damaged_commute_data['flow_c'] = damaged_commute_data['FLOW'] * (1-damaged_commute_data['damage_ratio_O']) * (1-damaged_commute_data['damage_ratio_D'])
    damaged_commute_data['flow_c'] = damaged_commute_data['flow_c'].astype(int)
    # damaged_commute_data['flow_hc'] = damaged_commute_data['FLOW'] * (1-0.5*damaged_commute_data['damage_ratio_O']) * (1-0.5*damaged_commute_data['damage_ratio_D'])
    # damaged_commute_data['flow_hc'] = damaged_commute_data['flow_hc'].astype(int)
    # damaged_commute_data['diff'] = damaged_commute_data['flow_n']-damaged_commute_data['flow_c']
    # damaged_commute_data['diff_ratio'] = damaged_commute_data['diff'] / damaged_commute_data['flow_n']
    # display(damaged_commute_data.head(0))
    # plt.hist(damaged_commute_data.loc[damaged_commute_data['diff_ratio']>-0.0, 'diff_ratio'], bins=100)
    # plt.show()
    print('Scen eq{}_bld{}_pip{}'.format(eq_i, building_threshold, pipe_threshold), damaged_commute_data.shape, np.sum(damaged_commute_data['flow_c']))
    return damaged_commute_data

def chunks(l, div):
    # Yield successive n-sized chunks from l.
    n = int(np.ceil(len(l)/div))
    for i in range(0, len(l), n):
        yield l[i:i + n]

def od_and_output(eq_i, building_threshold, pipe_threshold, damaged_commute_data=None, tract_nodes_dict=None):
    ### tract-to-tract OD to node-to-node OD
    display(damaged_commute_data[['OFIPS', 'DFIPS', 'flow_c']].head(2))
    print('Scen eq{}_bld{}_pip{}, total OD pairs {} under damage'.format(eq_i, building_threshold, pipe_threshold, np.sum(damaged_commute_data['flow_c'])))

    for scenario in ['c']:
        node_od_list = []
        unsuccessful_od = []
        for tract_od in damaged_commute_data.itertuples():
            # origin_tract = int(getattr(tract_od, 'OFIPS'))
            # destin_tract = int(getattr(tract_od, 'DFIPS'))
            origin_tract = getattr(tract_od, 'OFIPS')
            destin_tract = getattr(tract_od, 'DFIPS')
            tract_od_flow = int(getattr(tract_od, 'flow_{}'.format(scenario)))
            try:
                node_o = random.choices(tract_nodes_dict[origin_tract], k=tract_od_flow)
                node_d = random.choices(tract_nodes_dict[destin_tract], k=tract_od_flow)
                node_od_list += list(zip(node_o, node_d))
            except KeyError:
                # print(origin_tract, destin_tract, tract_od_flow)
                unsuccessful_od.append((origin_tract, destin_tract))
            except IndexError:
                unsuccessful_od.append((origin_tract, destin_tract))
        random.shuffle(node_od_list)
        print('      Scenario eq{}_bld{}_pip{}: {}/{} tract OD failed to match to nodes'.format(eq_i, building_threshold, pipe_threshold, len(unsuccessful_od), damaged_commute_data.shape[0]))

        div_node_od_super_list = list(chunks(node_od_list, 3))
        chunk_num = 0
        agent_id_shift = 0
        for div_node_od_list in div_node_od_super_list:
            div_node_od_df = pd.DataFrame(div_node_od_list, columns=['O', 'D'])
            div_node_od_df['agent_id'] = agent_id_shift
            div_node_od_df['agent_id'] += range(div_node_od_df.shape[0])
            agent_id_shift += div_node_od_df.shape[0]
            div_node_od_df[['agent_id', 'O', 'D']].to_csv('ctpp/od_eq{}_bld{}_pip{}-{}.csv'.format(eq_i, building_threshold, pipe_threshold, chunk_num), index=False)
            chunk_num += 1

# pipeline_scenario=0
for eq_i in range(20):
    damaged_tract_data = ba_census_tracts_gdf.merge(tract_building_damage_df, how='left', on='GEO_ID').merge(tract_pipe_damage_df, how='left', on='GEO_ID')
    damaged_commute_data = calculate_flow_after_damage(eq_i, building_threshold, pipe_threshold, damaged_tract_data=damaged_tract_data)
    # break
    od_and_output(eq_i, building_threshold, pipe_threshold, damaged_commute_data=damaged_commute_data, tract_nodes_dict=tract_nodes_dict)

Scen eq0_bld5_pip100 (100158, 8) 2046655


,OFIPS,DFIPS,flow_c
0,06001400100,06001400500,20
1,06001400100,06001401000,10


Scen eq0_bld5_pip100, total OD pairs 2046655 under damage
      Scenario eq0_bld5_pip100: 27/100158 tract OD failed to match to nodes
Scen eq1_bld5_pip100 (100158, 8) 2046655


,OFIPS,DFIPS,flow_c
0,06001400100,06001400500,20
1,06001400100,06001401000,10


Scen eq1_bld5_pip100, total OD pairs 2046655 under damage
      Scenario eq1_bld5_pip100: 27/100158 tract OD failed to match to nodes
Scen eq2_bld5_pip100 (100158, 8) 2046655


...

Scen eq17_bld5_pip100, total OD pairs 2046655 under damage
      Scenario eq17_bld5_pip100: 27/100158 tract OD failed to match to nodes
Scen eq18_bld5_pip100 (100158, 8) 2046655


,OFIPS,DFIPS,flow_c
0,06001400100,06001400500,20
1,06001400100,06001401000,10


Scen eq18_bld5_pip100, total OD pairs 2046655 under damage
      Scenario eq18_bld5_pip100: 27/100158 tract OD failed to match to nodes
Scen eq19_bld5_pip100 (100158, 8) 2046432


,OFIPS,DFIPS,flow_c
0,06001400100,06001400500,20
1,06001400100,06001401000,10


Scen eq19_bld5_pip100, total OD pairs 2046432 under damage
      Scenario eq19_bld5_pip100: 27/100158 tract OD failed to match to nodes


In [11]:
### flow geometry
ba_tracts_centroid_gdf = ba_census_tracts_gdf.copy()
ba_tracts_centroid_gdf['c_x'] = ba_census_tracts_gdf['geometry'].centroid.x
ba_tracts_centroid_gdf['c_y'] = ba_census_tracts_gdf['geometry'].centroid.y

damaged_commute_data_geom = damaged_commute_data[['OFIPS', 'DFIPS', 'flow_n', 'flow_c', 'flow_hc']].copy().merge(
    ba_tracts_centroid_gdf[['FIPS', 'c_x', 'c_y']], how='left', left_on='OFIPS', right_on='FIPS').merge(
    ba_tracts_centroid_gdf[['FIPS', 'c_x', 'c_y']], how='left', left_on='DFIPS', right_on='FIPS', suffixes=['_O', '_D'])
damaged_commute_data_geom['geometry'] = damaged_commute_data_geom.apply(lambda x: 'LINESTRING ({} {}, {} {})'.format(x['c_x_O'], x['c_y_O'], x['c_x_D'], x['c_y_D']), axis=1)
damaged_commute_data_geom[['OFIPS', 'DFIPS', 'flow_n', 'flow_c', 'flow_hc', 'geometry']].to_csv('commute_flow_pattern.csv', index=False)
